# Flower classifier 

*Author: Baccega Sandro*

In this notebook we will classify Oxford's `102 Category Flower Dataset`.

---

## Imports and constants

In [50]:
from scipy.io import loadmat
import pandas as pd
import numpy as np
import torch
import shutil
import time
import json
import os
import copy
import matplotlib.pyplot as plt
# import seaborn as sns
import numpy as np
from PIL import Image
from collections import OrderedDict
from torch import nn, optim
import torch.utils.data as data
from torch.utils.data import Dataset
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torchvision import datasets, models, transforms

RAW_IMAGE_LABELS_MAT_FILE = "assets/imagelabels.mat"
RAW_DATASET_IMAGES_FOLDER = "assets/jpg"
RAW_SEGMENTED_IMAGES_FOLDER = "assets/segmim"

DATASET_IMAGES_FOLDER = "data"

# Setting seed
torch.manual_seed(151836)

# Set device to use for computations
device = (
    "cuda"
    if torch.cuda.is_available()
    else ("mps" if torch.backends.mps.is_available() else "cpu")
)

print(f"-----\nPyTorch version: {torch.__version__}\nDevice: {device}\n-----")


-----
PyTorch version: 1.13.0.dev20220608
Device: mps
-----


## Creating sorted data folder

In [52]:
# If data folder exists, do not create images folder
if not os.path.isdir(DATASET_IMAGES_FOLDER):
    print("Creating data folder")        
    os.mkdir(DATASET_IMAGES_FOLDER)
    df = pd.DataFrame()
    df['Image'] = sorted(os.listdir(RAW_DATASET_IMAGES_FOLDER))
    df['Category'] = loadmat(RAW_IMAGE_LABELS_MAT_FILE)['labels'][0] - 1
    df['Category'] = df['Category'].astype(str)

    groups = df.groupby('Category')['Image'].apply(list)

    for category, images in groups.items():
        os.mkdir('{}/{}'.format(DATASET_IMAGES_FOLDER, category))
        for image in images:
            shutil.copyfile('{}/{}'.format(RAW_DATASET_IMAGES_FOLDER, image), '{}/{}/{}'.format(DATASET_IMAGES_FOLDER,category,image))

    print("Done")
else:
    print("Skipping")        



    

Skipping


## Creating datasets

In [15]:
class TransformDataset(Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform
        
    def __getitem__(self, index):
        x, y = self.subset[index]
        if self.transform:
            x = self.transform(x)
        return x, y
        
    def __len__(self):
        return len(self.subset)


dataset = datasets.ImageFolder(DATASET_IMAGES_FOLDER)

# Get 80% of dataset 
train_set_size = int(len(dataset) * 0.8)
valid_set_size = len(dataset) - train_set_size

untransformed_train_dataset, untransformed_validation_dataset = data.random_split(dataset, [train_set_size, valid_set_size])

train_dataset = TransformDataset(
    untransformed_train_dataset, transform=transforms.Compose([
        transforms.RandomRotation(45),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ])
)
validation_dataset = TransformDataset(
    untransformed_validation_dataset, transform=transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ])
)